In [1]:
# 采用全量数据训练embedding 只训练用户属性去表达广告属性，且广告属性以'N'补全所有句子（没有点击行为）
import pandas as pd
import numpy as np
import gc
from base import Cache
from tqdm import tqdm
from gensim.models import Word2Vec
import sys
from multiprocessing import Pool
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 5000)

# # log
# class Logger(object):
#     def __init__(self, fileN="Default.log"):
#         self.terminal = sys.stdout
#         self.log = open(fileN, "a", encoding='utf-8')

#     def write(self, message):
#         self.terminal.write(message)
#         self.log.write(message)

#     def flush(self):
#         pass
# sys.stdout = Logger("zlh0918log.txt")

def reduce_mem(df, use_float16=False):
    start_mem = df.memory_usage().sum() / 1024**2
    tm_cols = df.select_dtypes('datetime').columns
    for col in df.columns:
        if col in tm_cols:
            continue
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
                        np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
                        np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
                        np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
                        np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(
                        np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
                        np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    
def gen_list_df(data_):
    data = data_.copy()
#     onehotlist = ['age','city_rank','career','gender']
    feature1,feature2 = list(data.columns)[-2],list(data.columns)[-1]# 主键,emb字段
    print(f'{feature1},{feature2} start!')
    set1 = set(data.query('label<2')[feature1]).intersection(set(data.query('label==2')[feature1]))
    set2 = set(data.query('label<2')[feature2]).intersection(set(data.query('label==2')[feature2]))
    data.loc[~data[feature1].isin(set1),feature1]=-1
    data.loc[~data[feature2].isin(set2),feature2]=-1# unk 词input不在字典里的词都是 -1 每点击过1的词 和没见过的词都用-1替代生成embedding
    setfeature1 = pd.DataFrame(list(set(data[feature1])),columns=[feature1])# 所有feature1
    list_df=data.query('label==1').groupby(feature1)[feature2].apply(lambda x: list(map(str,list(x)))).\
            rename(f'list_{feature2}').reset_index()# 只有有点击记录的feature1
    list_df = setfeature1.merge(list_df,on=feature1,how='left')
    list_df[f'list_{feature2}']=list_df[f'list_{feature2}'].map(lambda x:x if isinstance(x,list) else ['N'])
    list_df_list = list_df[f'list_{feature2}'].values.tolist()
    print('w2v start!')
    model = Word2Vec(
    list_df_list,
    size=64,
    window=10000,# 增加了10倍
    workers=5,
    min_count=1,
    sg=0,
    hs=0,
    negative=5,
    iter=5,
    seed=0)
    
    emb_matrix = []
    for index,seq in enumerate(list_df_list):
        vec = []
        for w in seq:
            if w in model.wv:
                vec.append(model.wv[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * 64)
            
    emb_matrix = np.array(emb_matrix,dtype=np.float32)
    # 广告属性字典
    dict_feature1 = {}
    for index,key in enumerate(list(list_df[feature1])):
        dict_feature1[key]=emb_matrix[index,:]# feature1的字典 int
    # 用户属性字典
    dict_feature2 = {}
    for index,key in enumerate(list(model.wv.vocab.keys())):
        if key!='N':
            dict_feature2[int(key)]=model.wv[key]# feature2的字典 也做int
    dict_key_value = {}
    dict_key_value['key']=dict_feature1
    dict_key_value['value']=dict_feature2
    # 保存一个pkl 字典 key为广告属性emb字典 value为用户属性字典
    Cache.cache_data(dict_key_value, nm_marker=f'EMB_TARGET_DICT_{feature1}__{feature2}_w2v__All')
    print(emb_matrix.shape,list_df.shape)
    print(f'{feature1},{feature2}done!')

from multiprocessing import Pool
if __name__ == '__main__':
#     print('start!')
    data = Cache.reload_cache('CACHE_dataall0831.pkl')
    data = data.reset_index(drop=True).reset_index()
    # 修补缺失值
    sparse_features=['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags', 'app_first_class', 'app_second_class', 'city', 'device_name', 'career', 'gender', 'net_type', 'residence', 'emui_dev', 'indu_name', 'age', 'city_rank']
    dense_features=['his_app_size', 'his_on_shelf_time', 'app_score', 'device_size', 'list_time', 'device_price', 'up_life_duration', 'up_membership_grade', 'membership_life_duration', 'consume_purchase', 'communication_avgonline_30d']

    for var in sparse_features:
        mode_num = data[var].mode()[0]
        shape_null = data.query('{}==-1'.format(var)).shape[0]
        print('process sparse int: ',var, 'fillna: ',mode_num, 'fillna_shape: ',shape_null)
        if shape_null>0:
            data.loc[data[var]==-1,var] = mode_num
            data[var] = data[var].astype(np.int8)
    
    for var in dense_features:
        mode_num = int(data[var].mean())
        shape_null = data.query('{}==-1'.format(var)).shape[0]
        print('process dense int: ',var, 'fillna: ',mode_num, 'fillna_shape: ',shape_null)
        if shape_null>0:
            data.loc[data[var]==-1,var] = mode_num
            data[var] = data[var].astype(int)
    data = reduce_mem(data, use_float16=False)
    Cache.cache_data(data, nm_marker='dataall0831_fillna')
    print('start!')
    data = Cache.reload_cache('CACHE_dataall0831_fillna.pkl')
#     gc.collect()
#     data['slot_id_net_type'] = data['slot_id'].values*10+data['net_type'].values# 作为场景交叉项 直接onehot
    print(data.shape)
    data['label']=data['label'].fillna(2).astype(int)# mask
    gc.collect()
    data['slot_id_net_type'] = data['slot_id'].values*10+data['net_type'].values# 作为场景交叉项 直接onehot
    # 生成一个emb matrix
    le = LabelEncoder()
    data['slot_id_net_type'] = le.fit_transform(data['slot_id_net_type']).astype(int)
    n_data_matrix= np.eye(data['slot_id_net_type'].nunique())[data['slot_id_net_type'].values]
    n_data_matrix = n_data_matrix.astype(np.int32)
    np.save('EMB_slot_id_net_type_embedding_matrix_All.npy',n_data_matrix)
    print('w2v start!')
    # 生成一个emb matrix
    user_fe_list = ['age','career','gender','city','device_name','residence','emui_dev','city_rank']# 'city_rank'
    item_fe_list = ['task_id','adv_id','adv_prim_id','tags','spread_app_id']
    poc_feature_list = []
    for i,vari in enumerate(user_fe_list):
        for j,varj in enumerate(item_fe_list):
#             poc_feature_list.append([vari,varj])# key只用广告id，比较多
            poc_feature_list.append([varj,vari])
    print(len(poc_feature_list))
    datalist = []
    for item in poc_feature_list:
        datalist.append(data[['uid','index','pt_d','label',item[0],item[1]]])# 拆分数据集
    with Pool(5) as p:
        result = p.map(gen_list_df, datalist)


start!
start!


[2020-09-22 01:32:57] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_dataall0831_fillna.pkl


(42907133, 38)
w2v start!
40
task_id,age start!
adv_prim_id,age start!
spread_app_id,age start!
adv_id,career start!
tags,career start!
w2v start!


[2020-09-22 01:54:52] - word2vec.py[line:1399] - INFO: collecting all words and their counts
[2020-09-22 01:54:52] - word2vec.py[line:1384] - INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2020-09-22 01:54:52] - word2vec.py[line:1407] - INFO: collected 7 word types from a corpus of 1445488 raw words and 50 sentences
[2020-09-22 01:54:52] - word2vec.py[line:1458] - INFO: Loading a fresh vocabulary
[2020-09-22 01:54:52] - word2vec.py[line:1482] - INFO: effective_min_count=1 retains 7 unique words (100% of original 7, drops 0)
[2020-09-22 01:54:52] - word2vec.py[line:1488] - INFO: effective_min_count=1 leaves 1445488 word corpus (100% of original 1445488, drops 0)
[2020-09-22 01:54:52] - word2vec.py[line:1547] - INFO: deleting the raw counts dictionary of 7 items
[2020-09-22 01:54:52] - word2vec.py[line:1550] - INFO: sample=0.001 downsamples 7 most-common words
[2020-09-22 01:54:52] - word2vec.py[line:1553] - INFO: downsampling leaves estimated 122769 word corpus

w2v start!


...

[2020-09-22 05:02:24] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 90.00% examples, 2107 words/s, in_qsize 6, out_qsize 0
[2020-09-22 05:02:24] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 4 more threads
[2020-09-22 05:02:25] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 96.25% examples, 2260 words/s, in_qsize 3, out_qsize 1
[2020-09-22 05:02:25] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 3 more threads
[2020-09-22 05:02:26] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 2 more threads
[2020-09-22 05:02:26] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 1 more threads
[2020-09-22 05:02:28] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 100.00% examples, 2746 words/s, in_qsize 0, out_qsize 1
[2020-09-22 05:02:28] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 0 more threads
[2020-09

(80, 64) (80, 2)
adv_prim_id,city_rankdone!
w2v start!


[2020-09-22 05:04:43] - word2vec.py[line:1399] - INFO: collecting all words and their counts
[2020-09-22 05:04:43] - word2vec.py[line:1384] - INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2020-09-22 05:04:43] - word2vec.py[line:1407] - INFO: collected 4 word types from a corpus of 1445488 raw words and 50 sentences
[2020-09-22 05:04:43] - word2vec.py[line:1458] - INFO: Loading a fresh vocabulary
[2020-09-22 05:04:43] - word2vec.py[line:1482] - INFO: effective_min_count=1 retains 4 unique words (100% of original 4, drops 0)
[2020-09-22 05:04:43] - word2vec.py[line:1488] - INFO: effective_min_count=1 leaves 1445488 word corpus (100% of original 1445488, drops 0)
[2020-09-22 05:04:43] - word2vec.py[line:1547] - INFO: deleting the raw counts dictionary of 4 items
[2020-09-22 05:04:43] - word2vec.py[line:1550] - INFO: sample=0.001 downsamples 4 most-common words
[2020-09-22 05:04:43] - word2vec.py[line:1553] - INFO: downsampling leaves estimated 88973 word corpus 

[2020-09-22 05:07:08] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 84.00% examples, 1314 words/s, in_qsize 8, out_qsize 0
[2020-09-22 05:07:18] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 86.00% examples, 1140 words/s, in_qsize 7, out_qsize 0
[2020-09-22 05:07:22] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 90.00% examples, 1184 words/s, in_qsize 5, out_qsize 0
[2020-09-22 05:07:22] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 4 more threads
[2020-09-22 05:07:27] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 94.00% examples, 1295 words/s, in_qsize 3, out_qsize 1
[2020-09-22 05:07:27] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 3 more threads
[2020-09-22 05:07:27] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 2 more threads
[2020-09-22 05:07:28] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 98.00% examples, 1495 

(50, 64) (50, 2)
spread_app_id,city_rankdone!


In [2]:
poc_feature_list

[['task_id', 'age'],
 ['adv_id', 'age'],
 ['adv_prim_id', 'age'],
 ['tags', 'age'],
 ['spread_app_id', 'age'],
 ['task_id', 'career'],
 ['adv_id', 'career'],
 ['adv_prim_id', 'career'],
 ['tags', 'career'],
 ['spread_app_id', 'career'],
 ['task_id', 'gender'],
 ['adv_id', 'gender'],
 ['adv_prim_id', 'gender'],
 ['tags', 'gender'],
 ['spread_app_id', 'gender'],
 ['task_id', 'city'],
 ['adv_id', 'city'],
 ['adv_prim_id', 'city'],
 ['tags', 'city'],
 ['spread_app_id', 'city'],
 ['task_id', 'device_name'],
 ['adv_id', 'device_name'],
 ['adv_prim_id', 'device_name'],
 ['tags', 'device_name'],
 ['spread_app_id', 'device_name'],
 ['task_id', 'residence'],
 ['adv_id', 'residence'],
 ['adv_prim_id', 'residence'],
 ['tags', 'residence'],
 ['spread_app_id', 'residence'],
 ['task_id', 'emui_dev'],
 ['adv_id', 'emui_dev'],
 ['adv_prim_id', 'emui_dev'],
 ['tags', 'emui_dev'],
 ['spread_app_id', 'emui_dev'],
 ['task_id', 'city_rank'],
 ['adv_id', 'city_rank'],
 ['adv_prim_id', 'city_rank'],
 ['tags',